# Statistical and Machine Learning Models for Fundamentalist Data

This notebook is a useful tool for investors interested in the Brazilian stock market. It integrates machine learning techniques and statistical models to analyze fundamentalist data of companies listed on the stock exchange. The aim is to provide in-depth analysis and facilitate investment decision-making, focusing on identifying opportunities and mitigating risks. It includes interactive visualizations and real-time updates, making it accessible and practical for both experienced investors and beginners.

## Initial Setup

### Install Packages

In [86]:
%pip install pandas -q
%pip install plotly -q
%pip install scikit-learn -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Import libs

In [87]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler, OneHotEncoder, normalize
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score as sil_score
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

### Create a file path default

In [88]:
file_path_book = str(Path(os.getcwd()).parent.parent / "data/book")

### Pandas Config

In [89]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Load data

In [90]:
df_fundamentals_book = pd.read_csv(file_path_book + "/fundamentals_book.csv")
df_fundamentals_book.head(5)

,ticker,long_name,sector,industry,market_cap,enterprise_value,total_revenue,profit_margins,operating_margins,dividend_rate,beta,ebitda,trailing_pe,forward_pe,volume,average_volume,fifty_two_week_low,fifty_two_week_high,price_to_sales_trailing_12_months,fifty_day_average,two_hundred_day_average,trailing_annual_dividend_rate,trailing_annual_dividend_yield,book_value,price_to_book,total_cash,total_cash_per_share,total_debt,earnings_quarterly_growth,revenue_growth,gross_margins,ebitda_margins,return_on_assets,return_on_equity,gross_profits,total_assets_approx,asset_turnover,earnings_growth_rate,dividend_payout_ratio,equity,debt_to_equity,roi,roce
0,ABCB4.SA,Banco ABC Brasil S.A.,Financial Services,Banks - Regional,4.265434e+09,1.477339e+10,1.941779e+09,0.41576,0.38826,1.56,0.679,0.000000e+00,4.069768,4.706601,92300.0,747165.0,15.85,21.99,2.196663,19.3382,18.14667,1.550,0.080687,24.518,0.785138,7.774306e+09,35.162,1.829846e+10,0.001,0.003,0.00000,0.00000,0.01530,0.15680,1.973086e+09,7.774306e+09,0.249769,0.1,155000.000000,-1.052416e+10,-1.738710,0.131438,0.000000
1,AGRO3.SA,BrasilAgro - Companhia Brasileira de Proprieda...,Consumer Defensive,Farm Products,2.466480e+09,2.912933e+09,1.249437e+09,0.21493,0.25031,3.21,0.432,2.648920e+08,9.450382,6.332481,298100.0,666692.0,22.29,32.71,1.974073,27.0106,25.58635,3.240,0.132029,22.237,1.113460,3.838370e+08,3.885,8.720750e+08,6.801,0.671,0.25252,0.21201,0.03839,0.12170,3.155040e+08,3.838370e+08,3.255124,680.1,47.640053,-4.882380e+08,-1.786168,0.428927,0.079343
2,RAIL3.SA,Rumo S.A.,Industrials,Railroads,4.228882e+10,5.524305e+10,1.031746e+10,0.07639,0.33544,0.07,0.227,4.522541e+09,54.309525,21.723810,5733400.0,14644522.0,16.21,24.44,4.098764,22.5852,20.95235,0.066,0.002993,8.334,2.736981,7.656040e+09,4.132,2.184320e+10,3.935,0.121,0.34493,0.43834,0.04252,0.05163,3.146360e+09,7.656040e+09,1.347623,393.5,1.677255,-1.418716e+10,-1.539646,0.186765,0.070519
3,ALPA3.SA,Alpargatas S.A.,Consumer Cyclical,Footwear & Accessories,5.309793e+09,6.482982e+09,4.022153e+09,-0.05671,-0.06434,0.40,0.571,-1.980000e+05,0.000000,0.000000,1100.0,3953.0,7.27,17.80,1.320137,8.7146,9.63540,0.000,0.000000,7.867,1.008008,4.142880e+08,0.614,1.550341e+09,0.000,-0.127,0.43246,-0.00005,-0.00910,-0.04153,1.968303e+09,4.142880e+08,9.708591,0.0,0.000000,-1.136053e+09,-1.364673,0.620417,-0.000029
4,ALPA4.SA,Alpargatas S.A.,Consumer Cyclical,Footwear & Accessories,5.350758e+09,6.395236e+09,4.022153e+09,-0.05671,-0.06434,0.43,0.571,-1.980000e+05,0.000000,14.555555,1132100.0,5605825.0,6.81,22.51,1.330322,8.3228,9.27290,0.000,0.000000,7.867,0.999110,4.142880e+08,0.614,1.550341e+09,0.000,-0.127,0.43246,-0.00005,-0.00910,-0.04153,1.968303e+09,4.142880e+08,9.708591,0.0,0.000000,-1.136053e+09,-1.364673,0.628930,-0.000029


## Models

#### Data standardization using StandardScaler

In [91]:
numeric_columns = df_fundamentals_book.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = df_fundamentals_book.drop(['ticker', 'long_name', 'industry'], axis='columns').copy(deep=True)
categorical_columns = categorical_columns.select_dtypes(include='object').columns

#### Transforming categorical features into numerical using OneHotEncoder

In [92]:
encoder = OneHotEncoder(sparse=False)
encoded_categorical = encoder.fit_transform(df_fundamentals_book[categorical_columns])

df_encoded = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_columns))
df_fundamentals_final = pd.concat([df_fundamentals_book.drop(categorical_columns, axis=1), df_encoded], axis=1)
df_fundamentals_final.columns = ['_'.join(col.lower().replace('-', '').split()) for col in df_fundamentals_final.columns]

df_fundamentals_final.head()

,ticker,long_name,industry,market_cap,enterprise_value,total_revenue,profit_margins,operating_margins,dividend_rate,beta,ebitda,trailing_pe,forward_pe,volume,average_volume,fifty_two_week_low,fifty_two_week_high,price_to_sales_trailing_12_months,fifty_day_average,two_hundred_day_average,trailing_annual_dividend_rate,trailing_annual_dividend_yield,book_value,price_to_book,total_cash,total_cash_per_share,total_debt,earnings_quarterly_growth,revenue_growth,gross_margins,ebitda_margins,return_on_assets,return_on_equity,gross_profits,total_assets_approx,asset_turnover,earnings_growth_rate,dividend_payout_ratio,equity,debt_to_equity,roi,roce,sector_basic_materials,sector_communication_services,sector_consumer_cyclical,sector_consumer_defensive,sector_energy,sector_financial_services,sector_healthcare,sector_industrials,sector_real_estate,sector_technology,sector_utilities
0,ABCB4.SA,Banco ABC Brasil S.A.,Banks - Regional,4.265434e+09,1.477339e+10,1.941779e+09,0.41576,0.38826,1.56,0.679,0.000000e+00,4.069768,4.706601,92300.0,747165.0,15.85,21.99,2.196663,19.3382,18.14667,1.550,0.080687,24.518,0.785138,7.774306e+09,35.162,1.829846e+10,0.001,0.003,0.00000,0.00000,0.01530,0.15680,1.973086e+09,7.774306e+09,0.249769,0.1,155000.000000,-1.052416e+10,-1.738710,0.131438,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,AGRO3.SA,BrasilAgro - Companhia Brasileira de Proprieda...,Farm Products,2.466480e+09,2.912933e+09,1.249437e+09,0.21493,0.25031,3.21,0.432,2.648920e+08,9.450382,6.332481,298100.0,666692.0,22.29,32.71,1.974073,27.0106,25.58635,3.240,0.132029,22.237,1.113460,3.838370e+08,3.885,8.720750e+08,6.801,0.671,0.25252,0.21201,0.03839,0.12170,3.155040e+08,3.838370e+08,3.255124,680.1,47.640053,-4.882380e+08,-1.786168,0.428927,0.079343,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,RAIL3.SA,Rumo S.A.,Railroads,4.228882e+10,5.524305e+10,1.031746e+10,0.07639,0.33544,0.07,0.227,4.522541e+09,54.309525,21.723810,5733400.0,14644522.0,16.21,24.44,4.098764,22.5852,20.95235,0.066,0.002993,8.334,2.736981,7.656040e+09,4.132,2.184320e+10,3.935,0.121,0.34493,0.43834,0.04252,0.05163,3.146360e+09,7.656040e+09,1.347623,393.5,1.677255,-1.418716e+10,-1.539646,0.186765,0.070519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,ALPA3.SA,Alpargatas S.A.,Footwear & Accessories,5.309793e+09,6.482982e+09,4.022153e+09,-0.05671,-0.06434,0.40,0.571,-1.980000e+05,0.000000,0.000000,1100.0,3953.0,7.27,17.80,1.320137,8.7146,9.63540,0.000,0.000000,7.867,1.008008,4.142880e+08,0.614,1.550341e+09,0.000,-0.127,0.43246,-0.00005,-0.00910,-0.04153,1.968303e+09,4.142880e+08,9.708591,0.0,0.000000,-1.136053e+09,-1.364673,0.620417,-0.000029,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ALPA4.SA,Alpargatas S.A.,Footwear & Accessories,5.350758e+09,6.395236e+09,4.022153e+09,-0.05671,-0.06434,0.43,0.571,-1.980000e+05,0.000000,14.555555,1132100.0,5605825.0,6.81,22.51,1.330322,8.3228,9.27290,0.000,0.000000,7.867,0.999110,4.142880e+08,0.614,1.550341e+09,0.000,-0.127,0.43246,-0.00005,-0.00910,-0.04153,1.968303e+09,4.142880e+08,9.708591,0.0,0.000000,-1.136053e+09,-1.364673,0.628930,-0.000029,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Feature Selection using PCA

In [93]:
n_components = 5
pca = PCA(n_components)
pca_result = pca.fit_transform(df_fundamentals_final.select_dtypes(include=['int', 'float64', 'number']))
explained_variance = pca.explained_variance_ratio_

variance_explained = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(variance_explained)

fig = px.bar(x=[f'PC{i+1}' for i in range(len(explained_variance))], y=explained_variance, labels={'x': 'Principal Component', 'y': 'Explained Variance'}, title='Explained Variance by Each Principal Component', color_discrete_sequence=['rgb(100, 195, 181)'], text=[f'{x:.2f}%' for x in explained_variance*100])
fig.update_traces(textposition='outside')
fig.update_layout(template='plotly_dark', font=dict(color='white'), height=500)
fig.show()

In [94]:
components = pca.components_
component_loadings = pd.DataFrame(components, columns=df_fundamentals_final.select_dtypes(include=['int', 'float64', 'number']).columns)

n_components_to_display = 5  
n_top_features = 20  
top_features_per_component = []

for i in range(n_components):
    component = component_loadings.iloc[i]
    top_features = component.abs().nlargest(n_top_features).index.tolist()
    top_features_per_component.append(top_features)

df_top_features = pd.DataFrame(top_features_per_component, index=[f'Component {i+1}' for i in range(n_components)])
df_top_features

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Component 1,enterprise_value,total_debt,equity,market_cap,total_assets_approx,total_cash,total_revenue,gross_profits,ebitda,average_volume,volume,asset_turnover,dividend_payout_ratio,dividend_rate,book_value,fifty_two_week_high,debt_to_equity,total_cash_per_share,two_hundred_day_average,fifty_two_week_low
Component 2,total_revenue,total_debt,market_cap,ebitda,gross_profits,enterprise_value,total_cash,total_assets_approx,equity,average_volume,volume,asset_turnover,dividend_payout_ratio,earnings_growth_rate,debt_to_equity,dividend_rate,book_value,total_cash_per_share,fifty_day_average,fifty_two_week_low
Component 3,total_assets_approx,total_cash,equity,market_cap,enterprise_value,total_revenue,gross_profits,total_debt,ebitda,average_volume,volume,dividend_payout_ratio,asset_turnover,debt_to_equity,fifty_two_week_high,earnings_growth_rate,total_cash_per_share,dividend_rate,two_hundred_day_average,fifty_day_average
Component 4,total_revenue,market_cap,enterprise_value,total_debt,equity,gross_profits,total_cash,total_assets_approx,ebitda,average_volume,volume,asset_turnover,dividend_payout_ratio,earnings_growth_rate,fifty_two_week_high,two_hundred_day_average,fifty_day_average,fifty_two_week_low,total_cash_per_share,trailing_pe
Component 5,gross_profits,ebitda,market_cap,total_revenue,enterprise_value,total_debt,equity,total_cash,total_assets_approx,average_volume,volume,asset_turnover,dividend_payout_ratio,earnings_growth_rate,dividend_rate,fifty_two_week_high,book_value,total_cash_per_share,forward_pe,fifty_two_week_low


In [95]:
pca.fit(df_fundamentals_final.select_dtypes(include=['int', 'float64', 'number']))
variance_explained = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(variance_explained)

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(1, len(variance_explained) + 1), y=cumulative_variance, mode='lines+markers', line=dict(color='rgb(100, 195, 181)'), name='Cumulative Variance Explained'))
fig.update_layout(title='Elbow Method for PCA Analysis', xaxis_title='Number of Principal Components', yaxis_title='Cumulative Variance Explained', xaxis=dict(tickmode='linear'), yaxis=dict(tickformat='.0%'), template='plotly_dark', font=dict(color='white')) 
fig.show()

#### Selecting the Ideal Number of Clusters

In [96]:
n_components = 4

In [97]:
pca = PCA(n_components)
principal_components = pca.fit_transform(df_fundamentals_final.select_dtypes(include=['int', 'float64', 'number']))
range_num_clusters = list(range(2, 10))
silhouette_scores = []
distortions = []

for num_cluster in range_num_clusters:

    kmeans = KMeans(n_clusters=num_cluster, max_iter=10_000, random_state=19051992)
    cluster_labels = kmeans.fit_predict(principal_components)
    silhouette_scores.append(sil_score(principal_components, cluster_labels))

    distortions.append(KMeans(n_clusters=num_cluster, max_iter=10_000, random_state=19051992).fit(df_fundamentals_final.select_dtypes(include=['int', 'float64', 'number'])).inertia_)

scaler = StandardScaler()
silhouette_scores_scaled = scaler.fit_transform(np.array(silhouette_scores).reshape(-1, 1)).flatten()
distortions_scaled = scaler.fit_transform(np.array(distortions).reshape(-1, 1)).flatten()

fig = go.Figure()
fig.add_trace(go.Scatter(x=range_num_clusters, y=silhouette_scores_scaled, mode='lines+markers', name='Silhouette Score'))
fig.add_trace(go.Scatter(x=range_num_clusters, y=distortions_scaled, mode='lines+markers', name='Distortions'))
fig.update_layout(title='Elbow Method for Determining the Optimal Number of Clusters (Normalized)', xaxis_title='Number of clusters', yaxis_title='Silhouette Score / Distortions (Normalized)', template='plotly_dark', font=dict(color='white'))
fig.show()

In [98]:
df_pca = pd.DataFrame(principal_components[:, :2], columns=['PC1', 'PC2'])
df_pca['cluster'] = cluster_labels

fig = px.scatter(df_pca, x='PC1', y='PC2', color='cluster', title='Visualization of Clusters with PCA', color_continuous_scale=px.colors.qualitative.Vivid)
fig.update_layout(template='plotly_dark', font=dict(color='white'))
fig.show()


In [99]:
def calcular_coesao(X, labels, centroids):
    coesao = 0
    for i in range(len(centroids)):
        cluster_points = X[labels == i]
        coesao += np.sum((cluster_points - centroids[i])**2)
    return coesao

def calcular_separacao(centroids):
    separacao = 0
    for i in range(len(centroids)):
        for j in range(i+1, len(centroids)):
            separacao += np.sum((centroids[i] - centroids[j])**2)
    return separacao

In [100]:
pca = PCA(n_components)
principal_components = pca.fit_transform(df_fundamentals_final.select_dtypes(include=['int', 'float64', 'number']))

tsne = TSNE(n_components=2, metric='cosine')
tsne_results = tsne.fit_transform(principal_components)

kmeans = KMeans(n_clusters=5, max_iter=10_000, random_state=19051992)
clusters = kmeans.fit_predict(tsne_results)

df_visualization = pd.DataFrame(tsne_results, columns=['TSNE1', 'TSNE2'])
df_visualization['cluster'] = clusters

labels = kmeans.labels_
centroids = kmeans.cluster_centers_

fig = go.Figure()
for i in range(5):
    cluster_points = df_visualization[df_visualization['cluster'] == i]
    fig.add_trace(go.Scatter(x=cluster_points['TSNE1'], y=cluster_points['TSNE2'], mode='markers', name=f'Cluster {i}', marker=dict(color=px.colors.qualitative.Vivid[i])))
fig.add_trace(go.Scatter(x=centroids[:, 0], y=centroids[:, 1], mode='markers', marker=dict(size=10, color='white'), name='Centroides'))
fig.update_layout(title='Visualization of Clusters with TSNE and Cosine Distance', template='plotly_dark', font=dict(color='white'))

cohesion = calcular_coesao(tsne_results, clusters, centroids)
separation = calcular_separacao(centroids)

fig.add_annotation(x=max(tsne_results[:, 0]), y=max(tsne_results[:, 1]), text=f'Cohesion: {cohesion:.2f}, Separation: {separation:.2f}', showarrow=False, yshift=10)
fig.show()

In [101]:
df_fundamentals_final['kmeans_cluster'] = clusters

#### Insights on Clustering

##### Companies per Cluster

In [102]:
number_companies_cluster = df_fundamentals_final.groupby('kmeans_cluster')['ticker'].count()
total_companies = number_companies_cluster.sum()
percentage_companies_cluster = (number_companies_cluster / total_companies) * 100
combined_text = [f"Companies: {count} - Percentage: {percent:.2f}%" for count, percent in zip(number_companies_cluster, percentage_companies_cluster)]

fig = go.Figure()
fig.add_trace(go.Bar(x=number_companies_cluster.index, y=number_companies_cluster.values, name='Number of Companies per Cluster', text=combined_text, marker=dict(color='rgb(100, 195, 181)')))
fig.update_traces(textposition='outside')
fig.update_layout(title='Number of Companies per Cluster', xaxis_title='Clusters', yaxis_title='Number of Companies', template='plotly_dark', font=dict(color='white'), height=550)
fig.show()

In [103]:
companies_clusters = df_fundamentals_final[['ticker', 'long_name', 'kmeans_cluster']].sort_values(by='kmeans_cluster', ascending=True).reset_index(drop=True)

fig = px.scatter(companies_clusters, x='ticker', y='kmeans_cluster', color='kmeans_cluster', hover_data=['long_name'], title='Companies in Clusters')
fig.update_traces(marker=dict(size=10))
fig.update_layout(title='Companies by Cluster', xaxis_title='Clusters', yaxis_title='Companies by Cluster', template='plotly_dark', font=dict(color='white'), height=550)
fig.show()

In [104]:
fig = px.treemap(df_fundamentals_final, path=['kmeans_cluster', 'ticker'], title='Treemap of Companies in Clusters')
fig.show()


##### Sectors by Cluster

In [105]:
sectors_cluster = df_fundamentals_final.groupby(['kmeans_cluster', df_fundamentals_book['sector']]).count()
sectors_cluster_reset = sectors_cluster.reset_index()
sectors_cluster_reset = sectors_cluster_reset[['kmeans_cluster', 'sector']]
heatmap_data = pd.crosstab(sectors_cluster_reset['sector'], sectors_cluster_reset['kmeans_cluster'])

text_data = [['' for _ in range(len(heatmap_data.columns))] for _ in range(len(heatmap_data.index))]

for sector_idx, sector in enumerate(heatmap_data.index):
    for cluster_idx, cluster in enumerate(heatmap_data.columns):
        if heatmap_data.loc[sector, cluster] == 0:
            text_data[sector_idx][cluster_idx] = f'The sector {sector} not within the cluster {cluster}'
        else:
            text_data[sector_idx][cluster_idx] = f'The sector {sector} is within the cluster {cluster}'

fig = go.Figure(data=go.Heatmap(z=heatmap_data, x=heatmap_data.columns, y=heatmap_data.index, colorscale='Viridis', text=text_data, hoverinfo='text', showscale=False))
fig.update_layout(title='Presence of Sectors by Cluster', xaxis_title='Cluster', yaxis_title='Setor', font=dict(color='white'), plot_bgcolor='black', paper_bgcolor='black')
fig.show()

#### Financial Assessment and Profitability

##### Liquidity and Reserves

In [106]:
liquidity_reserves = df_fundamentals_final.groupby(['kmeans_cluster'])[['profit_margins', 'operating_margins', 'ebitda', 'gross_profits']].mean()
liquidity_reserves = liquidity_reserves.reset_index()
liquidity_reserves

,kmeans_cluster,profit_margins,operating_margins,ebitda,gross_profits
0,0,0.215149,0.234825,1.960104e+10,3.476357e+10
1,1,0.655540,-0.770747,6.152639e+07,2.490367e+08
2,2,0.282780,0.256726,1.464984e+09,1.990848e+09
3,3,0.097528,0.158076,2.224860e+08,8.309553e+08
4,4,-0.105806,0.080784,1.394380e+09,2.538414e+09


In [112]:
fig = go.Figure()
fig.add_trace(go.Bar(x=liquidity_reserves['kmeans_cluster'], y=liquidity_reserves['profit_margins'], name='Average Profit Margins', marker=dict(color='rgb(100, 195, 181)'), text=[f'{x:.2f}%' for x in (liquidity_reserves['profit_margins']*100)]))
fig.update_traces(textposition='outside')
fig.update_layout(title='Average Profit Margins by Cluster', xaxis_title='KMeans Cluster', yaxis_title='Average Profit Margins', template='plotly_dark', font=dict(color='white'), height=550)
fig.show()

In [113]:
fig = go.Figure()
fig.add_trace(go.Bar(x=liquidity_reserves['kmeans_cluster'], y=liquidity_reserves['operating_margins'], name='Average Operating Margins', marker=dict(color='rgb(100, 195, 181)'), text=[f'{x:.2f}%' for x in (liquidity_reserves['operating_margins']*100)]))
fig.update_traces(textposition='outside')
fig.update_layout(title='Average Operating Margins by Cluster', xaxis_title='KMeans Cluster', yaxis_title='Average Profit Margins', template='plotly_dark', font=dict(color='white'), height=550)
fig.show()

In [123]:
def format_values(values):
    formatted = []
    for value in values:
        if value >= 1e9:
            formatted.append(f'R${value / 1e9:.2f} B')
        elif value >= 1e6:
            formatted.append(f'R${value / 1e6:.2f} M')
        else:
            formatted.append(f'R${value:.2f}')
    return formatted

In [125]:
formatted_ebitda = format_values(liquidity_reserves['ebitda'])

fig = go.Figure()
fig.add_trace(go.Bar(x=liquidity_reserves['kmeans_cluster'], y=liquidity_reserves['ebitda'], hovertext=formatted_ebitda, name='Average Ebitda', marker=dict(color='rgb(100, 195, 181)'), text=formatted_ebitda))
fig.update_traces(textposition='outside')
fig.update_layout(title='Average Ebitda by Cluster', xaxis_title='KMeans Cluster', yaxis_title='Average Ebitda', template='plotly_dark', font=dict(color='white'), height=550)
fig.show()

In [129]:
formatted_gross_profits = format_values(liquidity_reserves['gross_profits'])

fig = go.Figure()
fig.add_trace(go.Bar(x=liquidity_reserves['kmeans_cluster'], y=liquidity_reserves['gross_profits'], hovertext=formatted_gross_profits, name='Gross Profits', marker=dict(color='rgb(100, 195, 181)'), text=formatted_gross_profits))
fig.update_traces(textposition='outside')
fig.update_layout(title='Gross Profits by Cluster', xaxis_title='KMeans Cluster', yaxis_title='Average Gross Profits', template='plotly_dark', font=dict(color='white'), height=550)
fig.show()

In [111]:
df_fundamentals_book.columns

Index(['ticker', 'long_name', 'sector', 'industry', 'market_cap',
       'enterprise_value', 'total_revenue', 'profit_margins',
       'operating_margins', 'dividend_rate', 'beta', 'ebitda', 'trailing_pe',
       'forward_pe', 'volume', 'average_volume', 'fifty_two_week_low',
       'fifty_two_week_high', 'price_to_sales_trailing_12_months',
       'fifty_day_average', 'two_hundred_day_average',
       'trailing_annual_dividend_rate', 'trailing_annual_dividend_yield',
       'book_value', 'price_to_book', 'total_cash', 'total_cash_per_share',
       'total_debt', 'earnings_quarterly_growth', 'revenue_growth',
       'gross_margins', 'ebitda_margins', 'return_on_assets',
       'return_on_equity', 'gross_profits', 'total_assets_approx',
       'asset_turnover', 'earnings_growth_rate', 'dividend_payout_ratio',
       'equity', 'debt_to_equity', 'roi', 'roce'],
      dtype='object')